In [1]:
# Import public packages and functions
import os
import pandas as pd
import numpy as np
import sys
import json
from pathlib import Path
from scipy import signal

import seaborn as sns
import matplotlib.pyplot as plt
import scikit_posthocs as sp

import warnings
warnings.filterwarnings("ignore")

# inserting the lib folder to the compiler
sys.path.insert(0, './lib')
sys.path.insert(0, './utils/')

import utils_io, utils_coherence

from lib_data import DATA_IO

C:\Users\a.kaymak\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
PATH_CURR = os.path.abspath(os.curdir)    # current code
PATH      = (str(Path(PATH_CURR).parent)) # data repository: upper directory where datasets situated
fs        = 2048

# 1. Measure Baseline Coherence Between LFP and ECOG Channels

In [10]:
#baseline_coherence_LFP_ECOG = utils_coherence.extract_baseline_coherence_between_ECOG_LFP_channels()
#baseline_coherence_LFP_ECOG.to_pickle(DATA_IO.path_events + "coherence/ECOG_LFP_BASELINE_COHERENCE.pkl")
baseline_coherence_LFP_ECOG = pd.read_pickle(DATA_IO.path_events + "coherence/ECOG_LFP_BASELINE_COHERENCE.pkl")

# 2. Measure Events Coherence Between LFP and ECOG Channels